In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision.transforms import InterpolationMode
from tqdm import tqdm  # 导入 tqdm 库

In [2]:
def get_transform():
    transform = transforms.Compose([
        transforms.Resize(256, interpolation=InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform

In [3]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.annotations.iloc[idx]["isic_id"] + ".jpg")
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(int(self.annotations.iloc[idx]["target"]))

        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
mytransform = get_transform()
dataset = CustomImageDataset(csv_file="../data/train-metadata.csv", img_dir="../data/train-image/image", transform=mytransform)

/tmp/ipykernel_2186878/224981577.py:3: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  self.annotations = pd.read_csv(csv_file)


In [ ]:
def print_samples(dataset, num_samples=5):
    for i in range(num_samples):
        image, label = dataset[i]
        print(f'Sample {i+1}:')
        print(f'  Image shape: {image.shape}')
        print(f'  Label: {label}')
        image_pil = transforms.ToPILImage()(image)
        image_pil.show()

print_samples(dataset)

In [ ]:
def check_dataset(dataset):
    try:
        for i in range(len(dataset)):
            image, label = dataset[i]
            assert isinstance(image, torch.Tensor), "Image is not a torch.Tensor"
            assert image.shape == (3, 600, 600), f"Unexpected image shape: {image.shape}"
            assert isinstance(label, int), "Label is not an integer"
        print("Dataset verification passed.")
    except Exception as e:
        print(f"Dataset verification failed: {e}")

check_dataset(dataset)

In [5]:
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
for inputs, labels in train_loader:
    print(f'Batch shape: {inputs.shape}')
    print(f'Labels: {labels}')
    break
      # 只打印一个批次的形状和标签

Batch shape: torch.Size([32, 3, 224, 224])
Labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])


In [ ]:
import pandas as pd

# 读取CSV文件
csv_file = '../data/train-metadata.csv'
data = pd.read_csv(csv_file)

# 计算target列中1和0的数量
target_counts = data['target'].value_counts()

print("Count of each class in the 'target' column:")
print(target_counts)

In [6]:
model = EfficientNet.from_pretrained('efficientnet-b0')
num_classes = 2


Loaded pretrained weights for efficientnet-b0


In [ ]:
for name, module in model.named_modules():
    print(name)

In [7]:
model._fc = nn.Sequential(
    nn.Linear(model._fc.in_features, 512),  # 新添加的全连接层
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, num_classes)  # 最终分类层
)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for param in model.parameters():
    param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model._fc.parameters(), lr=0.001)
for param in model._fc.parameters():
    param.requires_grad = True

model = model.to(device)

In [ ]:
import torch
import gc

  # Replace `your_tensor` with the actual tensor variable
gc.collect()
torch.cuda.empty_cache()

In [9]:
# 训练模型
num_epochs = 10  # 设置总训练轮数
save_interval = 2  # 设置保存间隔

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    # 使用 tqdm 包装数据加载器以显示进度条
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    # 每10个epoch保存一次模型
    if (epoch + 1) % save_interval == 0:
        model_save_path = f'./trained_model/efficient7b_model_epoch_{epoch + 1}.pth'
        torch.save(model.state_dict(), model_save_path)
        print(f'Model saved to {model_save_path}')

Epoch 1/10:  51%|█████▏    | 6432/12534 [17:39<18:11,  5.59it/s]